Download NLTK libraries

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

Punction, sentence tokenizer, pos tagging, lemmatization process

In [17]:
import nltk
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

def punctionProcess(text):
    out = str(text).maketrans('','', string.punctuation)
    result = str(text).translate(out)
    
    return result

#punktTokenizer for sentence detection
def punktSentenceTokenizer(paragraph):
    sent_tokenizer = nltk.tokenize.PunktSentenceTokenizer()
    sentences = sent_tokenizer.tokenize(paragraph)
    updateSentence = solveSentenceTokenizer(sentences)
    if len(updateSentence) == 0:
        updateSentence = sentences

    return updateSentence


#POS Tagging
def posTag(text):
    txt = word_tokenize(text)
    pos = nltk.pos_tag(txt)

    return pos

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# Lemmatize with POS Tag
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


#Lemmatization process
lemmatizer=WordNetLemmatizer()
def lemmaProcess(sentence):       
    input_str=word_tokenize(sentence)
    txt = ''
    for word in input_str:
        txt += lemmatizer.lemmatize(word, get_wordnet_pos(word)) + ' ' #, get_wordnet_pos(word)
    sentence = str(txt).strip()

    return sentence


#developed solve for sentence tokenizer error
def solveSentenceTokenizer(sentences):
    firstStage = []
    ind = 0
    indList = []
    last = []
    #If next sentence first word's first letter is lower or numeric char, combine with sentences
    s = ''
    for sentence in sentences:
        if ind+1 < len(sentences):
            words = word_tokenize(sentences[ind+1])
            word = words[0]
            if str(word).islower() == True or str(word).isnumeric() == True:
                firstStage.append(sentence+ ' ' + sentences[ind+1])
                s += sentence+ ' ' + sentences[ind+1]
                last.append(ind+1)
                if ind+1 - last[len(last)-2] == 1:
                    sent = firstStage[len(firstStage)-2]
                    firstStage.remove(firstStage[len(firstStage)-2])
                    firstStage.remove(firstStage[len(firstStage)-1])
                    sent += ' ' + sentences[ind+1]
                    firstStage.append(sent)
                    s += sent
            else:
                firstStage.append(sentence)  
        else:
                firstStage.append(sentence) 
        ind += 1
    for ind in range(0, len(firstStage)-1):
        if str(firstStage[ind]).find(firstStage[ind+1]) != -1:
            indList.append(ind+1)
    indList.sort(reverse=True)
    for ind in indList:
        firstStage.remove(firstStage[ind])
    #If sentence containing max 2 word, combines with next sentences
    secondStage = []
    ind = 0
    for sentence in firstStage:
        words = word_tokenize(sentence) 
        if len(words) < 3:
            if  ind+1 < len(firstStage):
                secondStage.append(sentence + ' ' + firstStage[ind+1])
        else:
            if len(secondStage) > 0:
                if str(secondStage[len(secondStage)-1]).find(sentence) == -1:
                    secondStage.append(sentence)
            else:
                secondStage.append(sentence)
        ind += 1
    #Previous word is uppercase, but last word's first character is uppercase with max 3 length => combine with next sentences 
    lastSentences = []
    ind = 0
    for update in secondStage:
        words = word_tokenize(update) 
        if len(words) >= 2:
            word = words[len(words)-2]
            if len(lastSentences) != 0:
                if str(lastSentences[len(lastSentences)-1]).find(update) != -1:
                    ind += 1
                    continue
            if str(word[0]).isupper() == True and len(word) < 4 and len(words) >= 3:
                word = words[len(words)-3]
                if str(word[0]).islower() == True and len(secondStage) > ind+1:
                    lastSentences.append(update + ' ' + secondStage[ind+1])
                else:
                    lastSentences.append(update)
            else:
                lastSentences.append(update)
        ind += 1
    return lastSentences

#Adding StanfordOpenIE library

In [ ]:
!pip install wget
!pip install stanfordnlp
#!pip install neuralcoref==4.0

In [19]:
import os
import tempfile
from pathlib import Path
from subprocess import Popen
from sys import stderr
from zipfile import ZipFile
import wget
from spacy.lang.en import English 

In [20]:
class StanfordOpenIE:

    def __init__(self, core_nlp_version: str = '4.1.0'):
        self.install_dir = Path('~/.stanfordnlp_resources/').expanduser()
        self.install_dir.mkdir(exist_ok=True)
        if len([d for d in self.install_dir.glob('*') if d.is_dir()]) == 0:
            # No coreNLP directories. Let's check for ZIP archives as well.
            zip_files = [d for d in self.install_dir.glob('*') if d.suffix == '.zip']
            if len(zip_files) == 0:
                # No dir and no ZIP. Let's download it with the desired core_nlp_version.
                remote_url = 'https://nlp.stanford.edu/software/stanford-corenlp-{}.zip'.format(core_nlp_version)
                print('Downloading from %s.' % remote_url)
                output_filename = wget.download(remote_url, out=str(self.install_dir))
                print('\nExtracting to %s.' % self.install_dir)
            else:
                output_filename = zip_files[0]
            print('Unzip %s.' % output_filename)
            zf = ZipFile(output_filename)
            zf.extractall(path=self.install_dir)
            zf.close()
        target_dir = [d for d in self.install_dir.glob('*') if d.is_dir()][0]

        os.environ['CORENLP_HOME'] = str(self.install_dir / target_dir)
        from stanfordnlp.server import CoreNLPClient
        self.client = CoreNLPClient(annotators=['openie'], memory='8G')

    def annotate(self, text: str, properties_key: str = None, properties: dict = None, simple_format: bool = True):
        """
        :param (str | unicode) text: raw text for the CoreNLPServer to parse
        :param (str) properties_key: key into properties cache for the client
        :param (dict) properties: additional request properties (written on top of defaults)
        :param (bool) simple_format: whether to return the full format of CoreNLP or a simple dict.
        :return: Depending on simple_format: full or simpler format of triples <subject, relation, object>.
        """
        # https://stanfordnlp.github.io/CoreNLP/openie.html
        core_nlp_output = self.client.annotate(text=text, annotators=['openie'], output_format='json',
                                               properties_key=properties_key, properties=properties)
        if simple_format:
            triples = []
            for sentence in core_nlp_output['sentences']:
                for triple in sentence['openie']:
                    triples.append({
                        'subject': triple['subject'],
                        'relation': triple['relation'],
                        'object': triple['object']
                    })
            return triples
        else:
            return core_nlp_output

    def generate_graphviz_graph(self, text: str, png_filename: str = './out/graph.png'):
        """
       :param (str | unicode) text: raw text for the CoreNLPServer to parse
       :param (list | string) png_filename: list of annotators to use
       """
        entity_relations = self.annotate(text, simple_format=True)
        """digraph G {
        # a -> b [ label="a to b" ];
        # b -> c [ label="another label"];
        }"""
        graph = list()
        graph.append('digraph {')
        for er in entity_relations:
            graph.append('"{}" -> "{}" [ label="{}" ];'.format(er['subject'], er['object'], er['relation']))
        graph.append('}')

        output_dir = os.path.join('.', os.path.dirname(png_filename))
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        out_dot = os.path.join(tempfile.gettempdir(), 'graph.dot')
        with open(out_dot, 'w') as output_file:
            output_file.writelines(graph)

        command = 'dot -Tpng {} -o {}'.format(out_dot, png_filename)
        dot_process = Popen(command, stdout=stderr, shell=True)
        dot_process.wait()
        assert not dot_process.returncode, 'ERROR: Call to dot exited with a non-zero code status.'

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

    def __del__(self):
        self.client.stop()
        del os.environ['CORENLP_HOME']

# Demo Process

Triple extraction with OpenIE

Coreference Resolution

In [ ]:
!pip install spacy==2.1.0
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz

In [ ]:
!git clone https://github.com/huggingface/neuralcoref.git

In [ ]:
!pip install neuralcoref==4.0

In [ ]:
%cd neuralcoref/

!pip install -r requirements.txt
!pip install -e .

In [ ]:
!python -m spacy validate

In [ ]:
import spacy
import neuralcoref

coreference_model = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(coreference_model)

In [ ]:
doc1 = coreference_model('My sister has a dog. She loves him.')
print(doc1._.coref_clusters)

In [5]:
def coreferenceResolution(paragraph, method):
    try:
        try:
            while paragraph.index('(') != -1:
                lindx = paragraph.index('(')
                rindx = paragraph.index(')', lindx)
                paragraph = paragraph[:lindx].strip() + ' ' + paragraph[rindx+1:].lstrip()
        except:
            paragraph = paragraph       
        sentences = punktSentenceTokenizer(paragraph)
        newParagraph = ''
        for sent in sentences:
            newParagraph += str(sent[:-1]) + '.. '
        print(newParagraph)
        if method == 1: #HuggingFace                       
            doc = coreference_model(newParagraph)  # get the spaCy Doc (composed of Tokens)
            paragraph = doc._.coref_resolved
        newSentence = paragraph.split('.. ')
        print(paragraph)
        sentences = []
        paragraph = ''
        for sentence in newSentence:    
            paragraph += sentence[:1].capitalize() + sentence[1:] + '. '
            sentences.append(sentence[:1].capitalize() + sentence[1:] + '. ')
        return sentences, paragraph[:-3]
    except Exception as e:
        print('ErrorCorefRes:' + str(e))
        return paragraph, ""

In [6]:
def wordProcess(question):
    from nltk.corpus import stopwords 
    from nltk.tokenize import word_tokenize
    stop_words = stopwords.words('english') 
    word_tokens = word_tokenize(question)
    filtered_ques = [w for w in word_tokens if not w.lower() in stop_words]
    ansType = NERstatement(question)
    question = ''
    for q in filtered_ques:
        question += q + ' '
    question = question[:-1].strip()

    return question, ansType

In [11]:
def NERstatement(ques):
    ansType = ''
    if ques.find('who') != -1:
        ansType = 'PERSON'
    if ques.find('when') != -1:
        ansType = 'DATE'
    if ques.find('how many') != -1:
        ansType = 'QUANTITY'
    if ques.find('how much') != -1:
        ansType = 'QUANTITY, MONEY'
    if ques.find('where') != -1:
        ansType = 'GPE'
    pos = posTag(ques)
    if len(pos)> 2:
        for i in range(len(pos)-1):
            if pos[i][0] == 'what' or pos[i][0] == 'which' or pos[i][0] == 'where' or pos[i][0] == 'whose':
                if pos[i+1][0] == 'people' or pos[i+1][0] == 'person':
                    ansType = 'PERSON'
                if pos[i+1][0] == 'nationality' or pos[i+1][0] == 'religion' or pos[i+1][0] == 'politic':
                    ansType = 'NORP'
                if pos[i+1][0] == 'country' or pos[i+1][0] == 'city' or pos[i+1][0] == 'state':
                    ansType = 'GPE'
                if pos[i+1][0] == 'company' or pos[i+1][0] == 'agency' or pos[i+1][0] == 'institution':
                    ansType = 'ORG'
                if pos[i+1][0] == 'building' or pos[i+1][0] == 'airport' or pos[i+1][0] == 'highway' or pos[i+1][0] == ' bridge':
                    ansType = 'FAC'
                if pos[i+1][0] == 'location' or pos[i+1][0] == 'range':
                    ansType = 'LOC'
                if pos[i+1][0] == 'hurricane' or pos[i+1][0] == 'battle' or pos[i+1][0] == 'war':
                    ansType = 'EVENT'
                if pos[i+1][0] == 'song' or pos[i+1][0] == 'title':
                    ansType = 'WORK_OF_ART'
                if pos[i+1][0] == 'language':
                    ansType = 'LANGUAGE'
                if pos[i+1][0] == 'time' or pos[i+1][0] == 'date' or pos[i+1][0] == 'period' or pos[i+1][0] == 'duration' or pos[i+1][0] == 'year' or pos[i+1][0] == 'month' or pos[i+1][0] == 'day':
                    ansType = 'DATE'
                if pos[i+1][0] == 'percentage':
                    ansType = 'PERCENT'
                if pos[i+1][0] == 'measurement' or pos[i+1][0] == 'weight' or pos[i+1][0] == 'distance':
                    ansType = 'QUANTITY'
    return ansType

nlp_model = spacy.load('en_core_web_sm')
def NERdetection(sentence, ansType):
    sentence = nlp_model(sentence)
    labels = [x.label_ for x in sentence.ents]
    answer_cont = False
    if len(labels) != 0:
        for i in range(0, len(labels)):
            if ansType.find(labels[i]) != -1:
                answer_cont = True
                break
    return answer_cont, labels

In [36]:
def OpenIEforSentence(sentence, client):
    svoText = []
    print('OpenIE entered')
    for triple in client.annotate(sentence, properties={
    'timeout': '500000'}):
        svoText.append(str(triple))
    return svoText

In [37]:
def demo(ques, parag_examp, actual_ans, sel_sentence, no_answer = False, preprocess = False):     
    
    with StanfordOpenIE() as StanfordClient:
        svoList = []
        corefSent = ''
        corefParag = ''
        sub_rel_obj = ''
        processes = ''
        if no_answer == False:
            paragraph = parag_examp
            actual_answer = actual_ans
            sentence = sel_sentence
            question = ques.lower()
            paragraph = str(paragraph).replace('\n', '')
            totalFindAnswer = 0
            findAns = 0
            sentences = punktSentenceTokenizer(paragraph)
            sind = -1
            sPosition = 0
            try:
                for sent in sentences:
                    if str(sent).find(str(sentence).strip()) != -1:
                        sPosition = sind + 1
                        sind += 1
                        break
                    sind += 1    
                if sind == -1:
                    print('Finding sentence index error')
                else:
                    corefSent, corefParag = coreferenceResolution(parag_examp, 1)
                    print(corefSent, corefParag)
                    svoList = OpenIEforSentence(corefSent[sPosition], StanfordClient)
                    corefSent = corefSent[sPosition]
            except Exception as e:
                print("Error LocationCoref:" + str(e))

            answerList = []            
            try:
                for triple in svoList:                 
                    triple = str(triple).split(',')
                    subject = str(str(triple[0]).split(':')[1]).strip()[1:-1].lower()            
                    relation = str(' ' + str(str(triple[1]).split(':')[1]).strip()[1:-1] + ' ').lower()
                    object = str(str(triple[2]).split(':')[1]).strip()[1:-2].lower()
                    if preprocess:
                        subject, ansType = wordProcess(subject)
                        relation, ansType = wordProcess(relation)
                        object, ansType = wordProcess(object)
                    answer = ''
                    if str(question).find(subject) != -1 and str(question).find(relation) != -1:
                        if str(question).find(object) == -1:
                            answer = object
                            sub_rel_obj = triple
                    if str(question).find(object) != -1 and str(question).find(relation) != -1:
                        if str(question).find(subject) == -1:    
                            answer = subject
                            sub_rel_obj = triple
                    if str(question).find(object) != -1 and str(question).find(subject) != -1:
                        if str(question).find(relation) == -1:    
                            answer = relation
                            sub_rel_obj = triple
                    if answer == actual_answer:
                        totalFindAnswer += 1
                    if answer != '':
                        answerList.append(answer)       
            except Exception as e:
                print("Error searchSvo:" + str(e))

            if len(answerList) == 1 and findAns != totalFindAnswer:
                print("Triple found successfully! Answer: " + str(answerList[0]))  

        else: 
            paragraph = parag_examp
            question = ques.lower()              
            paragraph = str(paragraph).replace('\n', '')
            triples = []
            corefSent, corefParag = coreferenceResolution(parag_examp, 1)
            for sent in corefSent:
                svoList = OpenIEforSentence(sent, StanfordClient)
                for svo in svoList:
                    triples.append(svo)
            corefSent = ''
            svoList = triples
            question, answerType = wordProcess(question)
            processes += 'NER answer label: ' + answerType + '\n'
            answer_cont = False
            labels = []
            try:
                for triple in triples: 
                    triple = str(triple).split(',')
                    subject = str(str(triple[0]).split(':')[1]).strip()[1:-1]
                    relation = str(' ' + str(str(triple[1]).split(':')[1]).strip()[1:-1] + ' ')
                    object = str(str(triple[2]).split(':')[1]).strip()[1:-2]
                    if preprocess:
                        subject, ansType = wordProcess(subject)
                        relation, ansType = wordProcess(relation)
                        object, ansType = wordProcess(object)
                    if str(question).find(subject.lower()) != -1 and str(question).find(relation.lower()) != -1:
                        if answerType != '':
                            answer_cont, labels = NERdetection(object.strip(), answerType)
                            processes += str(labels) + '\n'
                        else: answer_cont = True
                    if str(question).find(object.lower()) != -1 and str(question).find(relation.lower()) != -1:   
                        if answerType != '':
                            answer_cont, labels = NERdetection(subject.strip(), answerType)
                            processes += str(labels) + '\n'
                        else: answer_cont = True
                    if str(question).find(object.lower()) != -1 and str(question).find(subject.lower()) != -1: 
                        if answerType != '':
                            answer_cont, labels = NERdetection(relation.strip(), answerType)
                            processes += str(labels) + '\n'
                        else: answer_cont = True
                    if answer_cont == True:
                        break
                if answer_cont == False:                    
                    print("Triple not found successfully!")  
      
            except Exception as e:
                print("Error searchSvo:" + str(e))


        StanfordClient.client.stop()
        return svoList, corefParag, corefSent, sub_rel_obj, processes

In [12]:
def defineQuestionTerm(preProcess):
    questionSearch = ''
    for terms in preProcess:
        for term in terms:
            if str(term).find('_') != -1:
                term = str(term).replace('_',' ')
            questionSearch += term + ','
    questionSearch = questionSearch[:-1]
    return questionSearch

Sorted all sentence according to QTP value

In [13]:
def findPath(rateList,textList):
    sortList = []
    sortTxt = ''
    for rate in rateList:
        value = max(rateList)
        ind = rateList.index(value)
        sortList.append(ind)
        rateList[ind] = 0
    for sort in sortList:
        sortTxt += textList[sort] + '\n'
    return sortTxt

Search question words in sentences with question term percentage(QTP)

In [14]:
def searchWord(text,paragraph,lemma):
    texts = []
    textSearch = ''
    path = ''
    try:
        if str(text).find(',') != -1:
            texts = str(text).split(',')
        else:
            texts.append(text)  
        par = paragraph          
        sentences = punktSentenceTokenizer(par)
        s_ind = 0
        rateList = []
        textList = []
        for sentence in sentences:         
            sent = punctionProcess(str(sentence))
            if lemma == True:
                sent = lemmaProcess(sent)
                sentence = sent
            c = 0
            qaText = [] 
            qTerm = []
            for i in range(0,len(texts)):
                texts[i] = str(texts[i]).strip()
                if str(sent.lower()).find(texts[i].lower()) != -1:
                    c += 1
                    qTerm.append(str(texts[i]))

            if c != 0:
                textSearch += 'Find question:' + str(qTerm) + ', QCount:' + str(c) + '\nQuestion rate:%' + str(c/len(texts)*100) + '\n' + str(s_ind+1) + '. sentence - ' + sentence + '\n\n'
                rateList.append(c/len(texts)*100)
                textList.append('Find question:' + str(qTerm) + ', QCount:' + str(c) + '\nQuestion rate:%' + str(c/len(texts)*100) + '\n' +  str(s_ind+1) + '. sentence - ' + sentence + '\n\n')
            s_ind += 1
        if len(rateList) != 0:
            path += findPath(rateList,textList)
    except Exception as e:
        print('SearchWord:' + str(e) + ', text:' + str(texts))
    return textSearch, path

In [15]:
paragraph_examples = 'Ege University is a public research university in İzmir, Turkey. It was founded in 1955. It is the first university to start courses in İzmir and the fourth oldest university in Turkey. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.'
answer_examples = ['1955', 'No Answer']
question_examples = ['When was Ege University founded?','What is the second oldest university in Turkey?']

paragraph = paragraph_examples

Question that has an answer

StanfordOpenIE has a timeout error in Google Colab, please you'll run your local repository.

In [38]:
question = question_examples[0]
answer = answer_examples[0]

question = punctionProcess(question)
X_list = word_tokenize(question) 
preProcess = [X_list]       
questionSearch = defineQuestionTerm(preProcess)  
textSearch, path = searchWord(questionSearch, paragraph, False)
texts = path.split('\n')

sel_sentence = ''
for txt in texts:
    if (str(txt).find(answer.lower()) != -1 or str(txt).find(answer) != -1):
        sentence = str(txt).split('sentence - ')
        sentence = str(sentence[1]).strip()
        sel_sentence = str(sentence).replace('\n', '').strip()
        break

print("Candidate sentence: " + sel_sentence)
svoList, corefParag, corefSent, sub_rel_obj, processes = demo(question_examples[0], paragraph, answer, sel_sentence, False, True)
print("Candidate sentence (coreference resolution): " + str(corefSent))
print("Related triples: " + str(sub_rel_obj))
svo = ''
ind = 1
for trp in svoList:
    trp = str(trp).replace('\'subject\'', 'subject')
    trp = str(trp).replace('\'relation\'', 'relation')
    trp = str(trp).replace('\'object\'', 'object')
    svo += str(ind) + '- ' + trp[1:-1] + '\n'
    ind += 1
print(processes)

Candidate sentence: It was founded in 1955.
Ege University is a public research university in İzmir, Turkey. It was founded in 1955. It is the first university to start courses in İzmir and the fourth oldest university in Turkey. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.
Ege University is a public research university in İzmir, Turkey.. It was founded in 1955.. It is the first university to start courses in İzmir and the fourth oldest university in Turkey.. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.. 
Ege University is a public research university in İzmir, Turkey.. Ege University was founded in 1955.. Ege University is the first university to start courses in İzmir and the fourth oldest university in Turkey.. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.. 
['Ege University is a public research university in İzmir, Turkey. ', 'Ege University was founded in 1955. ', 'Ege Uni

Question that has no answer

In [33]:
question = question_examples[1]
answer = answer_examples[1]

svoList, corefParag, corefSent, sub_rel_obj, processes = demo(question_examples[1], paragraph, '', '', True, False)
svo = ''
ind = 1
for trp in svoList:
    trp = str(trp).replace('\'subject\'', 'subject')
    trp = str(trp).replace('\'relation\'', 'relation')
    trp = str(trp).replace('\'object\'', 'object')
    svo += str(ind) + '- ' + trp[1:-1] + '\n'
    ind += 1
print(processes) 

Ege University is a public research university in İzmir, Turkey. It was founded in 1955. It is the first university to start courses in İzmir and the fourth oldest university in Turkey. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.
Ege University is a public research university in İzmir, Turkey.. It was founded in 1955.. It is the first university to start courses in İzmir and the fourth oldest university in Turkey.. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.. 
Ege University is a public research university in İzmir, Turkey.. Ege University was founded in 1955.. Ege University is the first university to start courses in İzmir and the fourth oldest university in Turkey.. Ege University in Bornova, a district of Izmir, the third largest city in Turkey.. 
OpenIE entered
Starting server with command: java -Xmx8G -cp /root/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer

PermanentlyFailedException: ignored